In [ ]:
# !pip install vit_keras
# !pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=41b00bd77c94e0a5e46269187b99b998fff14662e1552db3f6fc240652c844ed
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Reshape, GlobalAveragePooling2D, GlobalMaxPooling2D 
from tensorflow.keras.optimizers import Adam
from vit_keras import vit

# Define CNN 1 layers
cnn_input1 = Input(shape=(224, 224, 3))
cnn_layer1 = Conv2D(filters=32, kernel_size=3, activation='relu')(cnn_input1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)
cnn_layer1 = Conv2D(filters=64, kernel_size=3, activation='relu')(cnn_layer1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)
cnn_layer1 = Conv2D(filters=128, kernel_size=3, activation='relu')(cnn_layer1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)

# Define CNN 2 layers
cnn_input2 = Input(shape=(224, 224, 3))
cnn_layer2 = Conv2D(filters=32, kernel_size=3, activation='relu')(cnn_input2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)
cnn_layer2 = Conv2D(filters=64, kernel_size=3, activation='relu')(cnn_layer2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)
cnn_layer2 = Conv2D(filters=128, kernel_size=3, activation='relu')(cnn_layer2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)

# Concatenate CNN layers
concat_layer = Concatenate()([cnn_layer1, cnn_layer2])

# Pooling layers
pooled_resnet = GlobalMaxPooling2D()(concat_layer)

# Define ViT layers
vit_input1 = Input(shape=(224, 224, 3))
vit_layer1 = vit.vit_b16(
    image_size=224,
    classes = 2,
    activation='sigmoid',
    pretrained=True,
    include_top=True
)(vit_input1)
vit_output1 = Reshape((256,))(pooled_resnet)
flatten = Flatten()(vit_output1)
output = Dense(units=1, activation='sigmoid')(flatten)

# Define the model
model = tf.keras.Model(inputs=[cnn_input1, cnn_input2, vit_input1], outputs=output)
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.9/dist-packages/vit_keras/vit.py:139: UserWarning: Can only use pretrained_top with imagenet21k+imagenet2012 if classes = 1000. Setting manually.
  warnings.warn(


347502902/347502902 [==============================] - 3s 0us/step


/usr/local/lib/python3.9/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_12 (Conv2D)             (None, 222, 222, 32  896         ['input_1[0][0]']                
                                )                                                             

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Reshape
from tensorflow.keras.optimizers import Adam
from vit_keras import vit

# Define CNN 1 layers
cnn_input1 = Input(shape=(224, 224, 3))
cnn_layer1 = Conv2D(filters=32, kernel_size=3, activation='relu')(cnn_input1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)
cnn_layer1 = Conv2D(filters=64, kernel_size=3, activation='relu')(cnn_layer1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)
cnn_layer1 = Conv2D(filters=128, kernel_size=3, activation='relu')(cnn_layer1)
cnn_layer1 = MaxPooling2D(pool_size=2)(cnn_layer1)
cnn_layer1 = Flatten()(cnn_layer1)
cnn_output1 = Dense(units=64, activation='relu')(cnn_layer1)

# Define CNN 2 layers
cnn_input2 = Input(shape=(224, 224, 3))
cnn_layer2 = Conv2D(filters=32, kernel_size=3, activation='relu')(cnn_input2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)
cnn_layer2 = Conv2D(filters=64, kernel_size=3, activation='relu')(cnn_layer2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)
cnn_layer2 = Conv2D(filters=128, kernel_size=3, activation='relu')(cnn_layer2)
cnn_layer2 = MaxPooling2D(pool_size=2)(cnn_layer2)
cnn_layer2 = Flatten()(cnn_layer2)
cnn_output2 = Dense(units=64, activation='relu')(cnn_layer2)

# Define ViT layers
vit_input1 = Input(shape=(224, 224, 3))
vit_layer1 = vit.vit_b16(
    image_size=224,
    activation='sigmoid',
    pretrained=True,
    include_top=True
)(vit_input1)
vit_output1 = Reshape((1000,))(vit_layer1)

# Concatenate CNN and ViT layers
concat_layer = Concatenate()([cnn_output1, cnn_output2, vit_output1])
output = Dense(units=1, activation='sigmoid')(concat_layer)

# Define the model
model = tf.keras.Model(inputs=[cnn_input1, cnn_input2, vit_input1], outputs=output)